In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt

def parse_device_data_chunked(file_path, num_chirps_per_frame, num_rx, num_adc_samples, start_frame, num_frames_in_chunk):
    num_samples_per_frame = num_chirps_per_frame * num_rx * num_adc_samples
    start_idx = start_frame * num_samples_per_frame * 2  # Multiply by 2 for I and Q int16 samples
    num_elements = num_frames_in_chunk * num_samples_per_frame * 2  # Total int16 elements in the chunk
    
    # Access data for the chunk using memory mapping
    file_size = os.path.getsize(file_path)
    total_elements = file_size // 2  # Total int16 elements in the file
    
    # Access data for the chunk using memory mapping
    
    
    adc_data_int16 = np.memmap(file_path, dtype=np.int16, mode='r', offset=start_idx * 2, shape=(num_elements,))
    adc_data_chunk = adc_data_int16.reshape(-1, 2)
    
    # Convert to complex64
    adc_data_complex_chunk = np.empty(adc_data_chunk.shape[0], dtype=np.complex64)
    adc_data_complex_chunk.real = adc_data_chunk[:, 0].astype(np.float32)
    adc_data_complex_chunk.imag = adc_data_chunk[:, 1].astype(np.float32)
    
    # Reshape into [frames, chirps, rx, samples]
    adc_data_complex_chunk = adc_data_complex_chunk.reshape(
        num_frames_in_chunk, num_chirps_per_frame, num_rx, num_adc_samples
    )
    
    return adc_data_complex_chunk

def construct_virtual_array(device_data_chunk, num_frames_in_chunk, num_chirps_per_frame, num_rx, num_adc_samples, chirp_tx_mapping):
    virtual_data = np.zeros((num_frames_in_chunk, num_chirps_per_frame, num_rx, num_adc_samples), dtype=np.complex64)
    
    for chirp_idx in range(num_chirps_per_frame):
        mapping = chirp_tx_mapping[chirp_idx % 12]  # There are 12 unique chirps
        device = mapping['device']
        # Get data for the current chirp from the corresponding device
        virtual_data[:, chirp_idx, :, :] = device_data_chunk[device][:, chirp_idx, :, :]
    
    return virtual_data

def range_fft(virtual_data_chunk, num_adc_samples):
    # Apply window function (e.g., Hanning window)
    window = np.hanning(num_adc_samples)
    virtual_data_windowed = virtual_data_chunk * window[np.newaxis, np.newaxis, np.newaxis, :]
    
    # Perform FFT along the samples dimension
    range_profiles = np.fft.fft(virtual_data_windowed, n=num_adc_samples, axis=-1)
    return range_profiles

def doppler_fft(range_profiles_chunk, num_chirps_per_frame):
    num_frames, num_chirps, num_rx, num_range_bins = range_profiles_chunk.shape
    
    # Apply window function along the chirp dimension
    window = np.hanning(num_chirps_per_frame)
    window = window[np.newaxis, :, np.newaxis, np.newaxis]
    range_profiles_windowed = range_profiles_chunk * window
    
    # Perform FFT along the chirp dimension
    doppler_fft_size = num_chirps_per_frame
    doppler_spectrum = np.fft.fftshift(np.fft.fft(range_profiles_windowed, n=doppler_fft_size, axis=1), axes=1)
    
    return doppler_spectrum

# Radar configuration parameters
num_chirps_per_frame = 768   # Total chirps per frame
num_rx = 4                   # Number of RX antennas per device
num_adc_samples = 256        # ADC samples per chirp

# Chirp timing parameters
idle_time = 5e-6             # 5 μs
ramp_end_time = 40e-6        # 40 μs
Tc = idle_time + ramp_end_time  # Chirp duration in seconds
fd = 1 / Tc                  # Doppler sampling frequency

# Radar carrier frequency
fc = 77e9                    # 77 GHz
c = 3e8                      # Speed of light
wavelength = c / fc          # Radar wavelength

# Chirp to Device Mapping
chirp_tx_mapping = {
    0: {'device': 'slave3', 'tx': 'TX2'},
    1: {'device': 'slave3', 'tx': 'TX1'},
    2: {'device': 'slave3', 'tx': 'TX0'},
    3: {'device': 'slave2', 'tx': 'TX2'},
    4: {'device': 'slave2', 'tx': 'TX1'},
    5: {'device': 'slave2', 'tx': 'TX0'},
    6: {'device': 'slave1', 'tx': 'TX2'},
    7: {'device': 'slave1', 'tx': 'TX1'},
    8: {'device': 'slave1', 'tx': 'TX0'},
    9: {'device': 'master', 'tx': 'TX2'},
    10: {'device': 'master', 'tx': 'TX1'},
    11: {'device': 'master', 'tx': 'TX0'},
}

data_directory = 'data/capture_drone_steady'  # Replace with your data directory
devices = ['master', 'slave1', 'slave2', 'slave3']

# Determine total number of frames (replace with actual calculation or value)
total_frames = 1707  # Example value, replace with actual total frames

# Set chunk size (number of frames per chunk)
chunk_size = 100  # Adjust based on your memory capacity

# Calculate the number of chunks
num_chunks = (total_frames + chunk_size - 1) // chunk_size  # Ceiling division

doppler_spectrum_accum = None  # Initialize accumulator

for chunk_idx in range(num_chunks):
    print(f"Processing chunk {chunk_idx + 1}/{num_chunks}...")
    
    start_frame = chunk_idx * chunk_size
    end_frame = min((chunk_idx + 1) * chunk_size, total_frames)
    num_frames_in_chunk = end_frame - start_frame

    # Parse data for each device in the current chunk
    chunk_device_data = {}
    for device in devices:
        print(f"Processing data for {device}...")
        device_data_files = [f for f in os.listdir(data_directory) if f.startswith(device) and '_data.bin' in f]
        device_data_files.sort()  # Ensure files are in order
        
        file_path = os.path.join(data_directory, device_data_files[start_frame:end_frame])    
        
        chunk_device_data[device] = parse_device_data_chunked(
            file_path=file_path,
            num_chirps_per_frame=num_chirps_per_frame,
            num_rx=num_rx,
            num_adc_samples=num_adc_samples,
            start_frame=start_frame,
            num_frames_in_chunk=num_frames_in_chunk
        )

    # Construct virtual antenna array for the chunk
    virtual_data_chunk = construct_virtual_array(
        device_data_chunk=chunk_device_data,
        num_frames_in_chunk=num_frames_in_chunk,
        num_chirps_per_frame=num_chirps_per_frame,
        num_rx=num_rx,
        num_adc_samples=num_adc_samples,
        chirp_tx_mapping=chirp_tx_mapping
    )

    # Perform Range FFT
    range_profiles_chunk = range_fft(virtual_data_chunk, num_adc_samples)

    # Identify the range bin corresponding to the drone (only once)
    if chunk_idx == 0:
        range_magnitude = np.abs(range_profiles_chunk)
        range_magnitude_mean = np.mean(range_magnitude, axis=(0, 2, 3))  # Average over frames, RX antennas, samples
        range_profile = range_magnitude_mean

        plt.figure()
        plt.plot(range_profile)
        plt.title('Range Profile')
        plt.xlabel('Range Bin')
        plt.ylabel('Amplitude')
        plt.show()

        selected_range_bin = int(input("Enter the index of the range bin corresponding to the drone: "))

    # Perform Doppler FFT
    doppler_spectrum_chunk = doppler_fft(range_profiles_chunk, num_chirps_per_frame)

    # Extract Doppler spectrum for the selected range bin
    doppler_spectrum_selected = doppler_spectrum_chunk[:, :, :, selected_range_bin]
    doppler_spectrum_mean = np.mean(doppler_spectrum_selected, axis=2)  # Average over RX antennas
    doppler_spectrum_sum = np.sum(doppler_spectrum_mean, axis=0)        # Sum over frames in the chunk

    # Accumulate Doppler spectrum
    if doppler_spectrum_accum is None:
        doppler_spectrum_accum = doppler_spectrum_sum
    else:
        doppler_spectrum_accum += doppler_spectrum_sum

# Generate frequency and velocity axes
doppler_fft_size = num_chirps_per_frame
freq_bins = np.fft.fftshift(np.fft.fftfreq(doppler_fft_size, d=Tc))
velocity_bins = freq_bins * wavelength / 2

# Plot the Doppler Spectrum (Frequency vs. Amplitude)
amplitude_spectrum = np.abs(doppler_spectrum_accum)

plt.figure()
plt.plot(freq_bins, amplitude_spectrum)
plt.title('Doppler Spectrum (Frequency vs. Amplitude)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

# Optional: Plot the Doppler Spectrum (Velocity vs. Amplitude)
plt.figure()
plt.plot(velocity_bins, amplitude_spectrum)
plt.title('Doppler Spectrum (Velocity vs. Amplitude)')
plt.xlabel('Radial Velocity (m/s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()


Processing chunk 1/18...
Processing data for master...


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'